<a href="https://colab.research.google.com/github/ykitaguchi77/Laboratory_course/blob/master/12.%20web%E3%82%A2%E3%83%97%E3%83%AA%E3%82%92%E4%BD%9C%E3%81%A3%E3%81%A6%E3%81%BF%E3%82%88%E3%81%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Streamlit test app**

準備：ngrok (https://ngrok.com/) にサインアップして、auth_tokenを取得

In [ ]:
# 必要なパッケージのインストール
!pip install streamlit --q
!pip install pyngrok --q

import streamlit as st
import os
from pyngrok import ngrok

# シンプルなアプリの作成
with open('test_app.py', 'w') as f:
    f.write('''
import streamlit as st

def main():
    st.title('テストアプリ')
    st.write('これは動作確認用の簡単なアプリです')

    # カウンターの初期化
    if 'count' not in st.session_state:
        st.session_state.count = 0

    # ボタンを作成
    if st.button('カウントアップ'):
        st.session_state.count += 1

    # カウントを表示
    st.write(f'現在のカウント: {st.session_state.count}')

if __name__ == '__main__':
    main()
''')

# Streamlitアプリの起動
def run_streamlit():
    # ポート設定
    port = 8501

    # 既存のプロセスを終了
    os.system('kill -9 $(lsof -t -i:8501) 2>/dev/null')
    print("既存のプロセスを終了しました")

    # ngrokの設定
    ngrok.set_auth_token('set yout ngrok token')  # あなたのトークンを設定

    # 既存のトンネルを閉じる
    ngrok.kill()

    # Streamlitの起動
    os.system(f'streamlit run test_app.py --server.port {port} &')

    # ngrokトンネルの作成
    public_url = ngrok.connect(port, "http")
    print(f'\n公開URL: {public_url}')

if __name__ == '__main__':
    run_streamlit()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.2 MB/s eta 0:00:00
既存のプロセスを終了しました

公開URL: NgrokTunnel: "https://ccac-34-45-183-24.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
# 必要なパッケージのインストール
!pip install streamlit --q
!pip install pyngrok --q

import os
import sys
from google.colab import drive
import shutil
import streamlit as st
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
from pyngrok import ngrok

# Google Driveのマウントとモデルファイルのコピー
print("=== Google Driveのマウントを開始 ===")
drive.mount('/content/drive')

# モデルファイルのパスを設定
source_path = "/content/drive/MyDrive/AI_laboratory_course/classification.pth"
dest_path = "/content/classification.pth"

# モデルファイルのコピー処理
print("\n=== モデルファイルのコピーを実行 ===")
if os.path.exists(source_path):
    try:
        shutil.copy(source_path, dest_path)
        print(f"モデルファイルのコピーが成功しました")
        print(f"コピー元: {source_path}")
        print(f"コピー先: {dest_path}")
    except Exception as e:
        print(f"モデルファイルのコピー中にエラーが発生しました: {str(e)}")
        sys.exit(1)
else:
    print(f"モデルファイルが見つかりません: {source_path}")
    print("以下のパスを確認してください:")
    !ls -R "/content/drive/MyDrive/AI_laboratory_course"
    sys.exit(1)



# Streamlitアプリケーションの作成
with open('app.py', 'w') as f:
    f.write('''
import os
import streamlit as st
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

class FruitClassifier(nn.Module):
    def __init__(self):
        super(FruitClassifier, self).__init__()
        self.resnet = models.resnet18(pretrained=False)
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, 2)

    def forward(self, x):
        return self.resnet(x)

def load_model():
    model_path = '/content/classification.pth'
    print(f"モデルファイルの存在確認: {os.path.exists(model_path)}")

    if not os.path.exists(model_path):
        raise FileNotFoundError(f"モデルファイルが見つかりません: {model_path}")

    try:
        model = FruitClassifier()
        state_dict = torch.load(model_path, map_location=torch.device('cpu'))
        model.load_state_dict(state_dict)
        model.eval()
        print("モデルのロードが成功しました")
        return model
    except Exception as e:
        print(f"モデルのロード中にエラーが発生しました: {str(e)}")
        raise

def preprocess_image(image):
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    image = preprocess(image)
    return image.unsqueeze(0)

def main():
    st.title('🍎 リンゴ・イチゴ 判別アプリ 🍓')
    st.write('画像をアップロードして、リンゴかイチゴかを判定します')

    try:
        print("=== モデルのロードを開始 ===")
        model = load_model()
        classes = ["apple", "strawberry"]
        print("=== モデルのロードが完了 ===")

        uploaded_file = st.file_uploader("画像をアップロードしてください", type=['png', 'jpg', 'jpeg'])

        if uploaded_file is not None:
            print("=== 画像の処理を開始 ===")
            image = Image.open(uploaded_file)
            st.image(image, caption='アップロードされた画像', use_column_width=True)

            try:
                input_tensor = preprocess_image(image)
                print("画像の前処理が完了しました")

                with torch.no_grad():
                    output = model(input_tensor)
                    probabilities = torch.nn.functional.softmax(output[0], dim=0)
                    predicted_class = torch.argmax(probabilities).item()
                print("予測が完了しました")

                st.write('## 判定結果')
                st.write(f'この画像は「**{classes[predicted_class]}**」です')

                st.write('### 確率')
                for i, prob in enumerate(probabilities):
                    st.write(f'{classes[i]}: {prob.item()*100:.2f}%')
                    st.progress(prob.item())

            except Exception as e:
                st.error('画像の処理中にエラーが発生しました。')
                st.error(f'エラー詳細: {str(e)}')
                print(f"画像処理エラー: {str(e)}")

    except Exception as e:
        st.error('モデルの読み込み中にエラーが発生しました。')
        st.error(f'エラー詳細: {str(e)}')
        print(f"モデルロードエラー: {str(e)}")

if __name__ == '__main__':
    main()
''')

print("\n=== Streamlitアプリの起動とngrokの設定を開始 ===")

# Streamlitアプリの起動とngrokの設定
def run_streamlit_with_ngrok():
    # ポート設定
    port = 8501

    # 既存のプロセスを終了
    os.system('kill -9 $(lsof -t -i:8501) 2>/dev/null')
    print("既存のStreamlitプロセスを終了しました")

    # ngrokの設定
    print("\n=== ngrokの設定を開始 ===")
    ngrok.set_auth_token('set yout ngrok token')  # ここにあなたのngrokのauthトークンを設定してください

    # 既存のngrokトンネルを閉じる
    ngrok.kill()
    print("既存のngrokトンネルを終了しました")

    # Streamlitの起動
    print("\n=== Streamlitアプリを起動します ===")
    os.system(f'streamlit run app.py --server.port {port} &')

    # 新しいngrokトンネルを作成
    try:
        public_url = ngrok.connect(port, "http")
        print(f'\n=== アプリケーションの準備が完了しました ===')
        print(f'公開URL: {public_url}')
    except Exception as e:
        print(f"ngrokトンネルの作成中にエラーが発生しました: {str(e)}")

if __name__ == '__main__':
    run_streamlit_with_ngrok()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.1 MB/s eta 0:00:00
=== Google Driveのマウントを開始 ===
Mounted at /content/drive

=== モデルファイルのコピーを実行 ===
モデルファイルのコピーが成功しました
コピー元: /content/drive/MyDrive/AI_laboratory_course/classification.pth
コピー先: /content/classification.pth

=== Streamlitアプリの起動とngrokの設定を開始 ===
既存のStreamlitプロセスを終了しました

=== ngrokの設定を開始 ===
既存のngrokトンネルを終了しました

=== Streamlitアプリを起動します ===

=== アプリケーションの準備が完了しました ===
公開URL: NgrokTunnel: "https://b107-35-232-83-98.ngrok-free.app" -> "http://localhost:8501"
